# PDF - Parse RH Shareholder Presentation using column patterns

In [404]:
import pdfplumber
import pandas as pd

with pdfplumber.open('RH Q2 2022 Financial Results and Shareholder Letter.pdf') as pdf:
    page = pdf.pages[15]
    page.crop
    text = page.extract_text()

print(text)

 
RECONCILIATION OF GAAP NET INCOME TO ADJUSTED NET INCOME 
(Unaudited)  
                           
 
THREE MONTHS ENDED    SIX MONTHS ENDED 
   
        JULY 30,        JULY 31,        JULY 30,        JULY 31,   
  2022    2021    2022    2021       
 
    (in thousands)   
GAAP net income    $   122,275   $   226,746   $   322,986   $   357,402 
Adjustments (pre-tax):                                
Employer payroll taxes on option exercise(1)       —      —      11,717      — 
Non-cash compensation(2)       4,321      5,864      10,179      11,728 
Asset impairments(3)       2,231      7,354      8,154      7,354 
Professional fees(4)       285      —      7,469      — 
Compensation settlements(5)        3,483       —       3,483       — 
Recall accrual(6)        —       —       560       500 
Reorganization related costs(7)       —      449      —      449 
Other expenses:                                
Loss on extinguishment of debt(8)        23,462      3,166       169,578    

In [406]:
import re
table_data_pattern = re.compile(r'([A-Z\(][A-z\d\s\:\(\)-—]*?) [\$\s]*([\(\)—\d,]+) [\$\s]* ([\(\)—\d,]+) [\$\s]* ([\(\)—\d,]+) [\$\s]* ([\(\)—\d,]+)') #re.compile(r'^([A-Z][A-z\s\$-\)\(]*)')

for line in text.split('\n'):
    line = table_data_pattern.search(line)
    if line:
        print(line.group(0))
#         print(type(line))
        
# for line in text.split('\n'):
#     print(line)

GAAP net income    $   122,275   $   226,746   $   322,986   $   357,402
Employer payroll taxes on option exercise(1)       —      —      11,717      —
Non-cash compensation(2)       4,321      5,864      10,179      11,728
Asset impairments(3)       2,231      7,354      8,154      7,354
Professional fees(4)       285      —      7,469      —
Compensation settlements(5)        3,483       —       3,483       —
Recall accrual(6)        —       —       560       500
Reorganization related costs(7)       —      449      —      449
Loss on extinguishment of debt(8)        23,462      3,166       169,578      3,271
(Gain) loss on derivative instruments—net(9)        1,453      —       (1,724)     —
Amortization of debt discount(10)        —       5,865       —       11,846
Subtotal adjusted items        35,235       22,698       209,416       35,148
Impact of income tax items(11)        56,397      (305)      (107,029)     (3,256)
Share of equity method investments losses(12)        2,821 

In [346]:
table_subheading_pattern = re.compile(r'^[A-Z][A-z\(\)\d\s-]*:')
for line in text.split('\n'):
#     line = table_subheading_pattern.search(line)
    if table_subheading_pattern.match(line):
        print(line)

Adjustments (pre-tax):                                
Other expenses:                                


In [371]:
from collections import namedtuple

table_nmtuple = namedtuple('t_row', 
                           'category sub_category Three_mnths_July_2022 Three_mnths_July_2021 Six_mnths_July_2022 Six_mnths_July_2021'
                          )

In [397]:
import re
import numpy as np

table_data_list = []

table_data_pattern = re.compile(r'([A-Z\(][A-z\d\s\:\(\)-—]*?) [\$\s]*([\(\)—\d,]+) [\$\s]* ([\(\)—\d,]+) [\$\s]* ([\(\)—\d,]+) [\$\s]* ([\(\)—\d,]+)') #re.compile(r'^([A-Z][A-z\s\$-\)\(]*)')
table_subheading_pattern = re.compile(r'^[A-Z][A-z\(\)\d\s-]*:')
sub_category = np.nan

for line in text.split('\n'):
    line_sh = table_subheading_pattern.search(line)
    if table_subheading_pattern.match(line):
        sub_category = line
        print(sub_category)
    
    line = table_data_pattern.search(line)
    if line:
        category = line.group(1)
        print(category)
        Three_mnths_July_2022 = line.group(2)
        Three_mnths_July_2021 = line.group(3)
        Six_mnths_July_2022 = line.group(4)
        Six_mnths_July_2021 = line.group(5)
        table_data_list.append(table_nmtuple(category, sub_category, Three_mnths_July_2022, Three_mnths_July_2021, Six_mnths_July_2022, Six_mnths_July_2021))

GAAP net income
Adjustments (pre-tax):                                
Employer payroll taxes on option exercise(1)
Non-cash compensation(2)
Asset impairments(3)
Professional fees(4)
Compensation settlements(5)
Recall accrual(6)
Reorganization related costs(7)
Other expenses:                                
Loss on extinguishment of debt(8)
(Gain) loss on derivative instruments—net(9)
Amortization of debt discount(10)
Subtotal adjusted items
Impact of income tax items(11)
Share of equity method investments losses(12)
Adjusted net income(13)


In [398]:
fetced_df_GAAP_recon = pd.DataFrame(table_data_list)

In [400]:
fetced_df_GAAP_recon

,category,sub_category,Three_mnths_July_2022,Three_mnths_July_2021,Six_mnths_July_2022,Six_mnths_July_2021
0,GAAP net income,NaN,"122,275","226,746","322,986","357,402"
1,Employer payroll taxes on option exercise(1),Adjustments (pre-tax): ...,—,—,"11,717",—
2,Non-cash compensation(2),Adjustments (pre-tax): ...,"4,321","5,864","10,179","11,728"
3,Asset impairments(3),Adjustments (pre-tax): ...,"2,231","7,354","8,154","7,354"
4,Professional fees(4),Adjustments (pre-tax): ...,285,—,"7,469",—
5,Compensation settlements(5),Adjustments (pre-tax): ...,"3,483",—,"3,483",—
6,Recall accrual(6),Adjustments (pre-tax): ...,—,—,560,500
7,Reorganization related costs(7),Adjustments (pre-tax): ...,—,449,—,449
8,Loss on extinguishment of debt(8),Other expenses:,"23,462","3,166","169,578","3,271"
9,(Gain) loss on derivative instruments—net(9),Other expenses:,"1,453",—,"(1,724)",—


## Page 18

In [407]:
import pdfplumber
import pandas as pd

with pdfplumber.open('RH Q2 2022 Financial Results and Shareholder Letter.pdf') as pdf:
    page = pdf.pages[17]
    text = page.extract_text()

print(text)

 
RECONCILIATION OF DILUTED NET INCOME PER SHARE TO  
ADJUSTED DILUTED NET INCOME PER SHARE 
(Unaudited)  
                           
 
THREE MONTHS ENDED    SIX MONTHS ENDED 
   
       JULY 30,        JULY 31,        JULY 30,        JULY 31,   
  2022    2021    2022    2021      
 
Diluted net income per share    $   5.37   $   7.09   $   17.70   $   11.31 
Pro forma diluted net income per share(1)(2)(3)    $   5.43   $   7.65   $   18.18   $   12.17 
Per share impact of adjustments (pre-tax)(4):                              
Employer payroll taxes on option exercise       —      —      0.43      — 
Non-cash compensation       0.16      0.20      0.38      0.40 
Asset impairments        0.08       0.25       0.30       0.25 
Professional fees       0.01      —      0.28      — 
Compensation settlements       0.14      —      0.12      — 
Recall accrual        —       —       0.02       0.02 
Amortization of debt discount       —      0.20      —      0.40 
Loss on extinguishment of

In [416]:
import re

p_17_pattern = re.compile(r'([A-Z\(][A-z\d\s\:\(\)-—]*?) [\$\s]*([\(\)—\d,\.]+) [\$\s]* ([\(\)—\d,\.]+) [\$\s]* ([\(\)—\d,\.]+) [\$\s]* ([\(\)—\d,\.]+)')

for line in text.split('\n'):
    line = p_17_pattern.search(line)
    if line:
        print(line.group(1))


Diluted net income per share
Pro forma diluted net income per share(1)(2)(3)
Employer payroll taxes on option exercise
Non-cash compensation
Asset impairments
Professional fees
Compensation settlements
Recall accrual
Amortization of debt discount
Loss on extinguishment of debt(5)
(Gain) loss on derivative instruments—net
Subtotal adjusted items
Impact of income tax items(4)
Share of equity method investments losses(4)
Adjusted diluted net income per share(6)


In [419]:
p_17_sub_category_pattern = re.compile(r'^[A-Z][A-z\(\)\d\s-]*:')

for line in text.split('\n'):
    line = p_17_sub_category_pattern.search(line)
    if line:
        print(line.group(0))

Per share impact of adjustments (pre-tax)(4):


In [425]:
from collections import namedtuple

p17_table_collection = namedtuple('t_row', 
                                  'category, sub_category Three_mnths_July_2022 Three_mnths_July_2021 Six_mnths_July_2022 Six_mnths_July_2021')

In [438]:
p_17_sub_category_pattern = re.compile(r'^[A-Z][A-z\(\)\d\s-]*:')
p_17_pattern = re.compile(r'([A-Z\(][A-z\d\s\:\(\)-—]*?) [\$\s]*([\(\)—\d,\.]+) [\$\s]* ([\(\)—\d,\.]+) [\$\s]* ([\(\)—\d,\.]+) [\$\s]* ([\(\)—\d,\.]+)')

p_17_table_data_list =[]
sub_category = np.nan

for line in text.split('\n'):
    if p_17_sub_category_pattern.match(line):
        sub_category = line
    
    line = p_17_pattern.search(line)
    if line:
        category = line.group(1)
        Three_mnths_July_2022 = line.group(2)
        Three_mnths_July_2021 = line.group(3)
        Six_mnths_July_2022 = line.group(4)
        Six_mnths_July_2021 = line.group(5)
        p_17_table_data_list.append(p17_table_collection(category, sub_category, Three_mnths_July_2022, Three_mnths_July_2021, Six_mnths_July_2022, Six_mnths_July_2021))
        print(category)


Diluted net income per share
Pro forma diluted net income per share(1)(2)(3)
Employer payroll taxes on option exercise
Non-cash compensation
Asset impairments
Professional fees
Compensation settlements
Recall accrual
Amortization of debt discount
Loss on extinguishment of debt(5)
(Gain) loss on derivative instruments—net
Subtotal adjusted items
Impact of income tax items(4)
Share of equity method investments losses(4)
Adjusted diluted net income per share(6)


In [439]:
pd.DataFrame(p_17_table_data_list)

,category,sub_category,Three_mnths_July_2022,Three_mnths_July_2021,Six_mnths_July_2022,Six_mnths_July_2021
0,Diluted net income per share,NaN,5.37,7.09,17.70,11.31
1,Pro forma diluted net income per share(1)(2)(3),NaN,5.43,7.65,18.18,12.17
2,Employer payroll taxes on option exercise,Per share impact of adjustments (pre-tax)(4): ...,—,—,0.43,—
3,Non-cash compensation,Per share impact of adjustments (pre-tax)(4): ...,0.16,0.20,0.38,0.40
4,Asset impairments,Per share impact of adjustments (pre-tax)(4): ...,0.08,0.25,0.30,0.25
5,Professional fees,Per share impact of adjustments (pre-tax)(4): ...,0.01,—,0.28,—
6,Compensation settlements,Per share impact of adjustments (pre-tax)(4): ...,0.14,—,0.12,—
7,Recall accrual,Per share impact of adjustments (pre-tax)(4): ...,—,—,0.02,0.02
8,Amortization of debt discount,Per share impact of adjustments (pre-tax)(4): ...,—,0.20,—,0.40
9,Loss on extinguishment of debt(5),Per share impact of adjustments (pre-tax)(4): ...,—,0.10,—,0.11
